# Yiteng Long (Simon)'s Replication: 
SMM estimation in DellaVigna, List, Malmendier and Rao, 2017, "Voting To Tell Others"

In [86]:

import numpy as np
import pandas as pd
from scipy import optimize
from scipy.io import loadmat
from scipy.linalg import block_diag
import time
import pickle
import os
# Import the empirical moments 
# I add baseline turnnout rate and flatten the imported data
dt_empmoments = loadmat('d:/Users/29457/Desktop/26Fall/Coding Sample/Moments.mat')                         
emp_moments = [dt_empmoments["Moments"]]
emp_moments = [item for sublist in emp_moments for item in sublist]       
emp_moments.append(np.array([0.6000]))                                    
emp_moments = np.ndarray.flatten(np.array(emp_moments))        

# Import empirical var-cov matrix; 101 moments
# W is the inverse, used as weighting matrix in SMM
emp_moments_varcov =  dt_empmoments["VCcontrol"]                          
emp_moments_varcov = block_diag(emp_moments_varcov,np.diag([0.0109**2]))  
W = np.linalg.inv(np.diag(np.diag(emp_moments_varcov)))                  

In [87]:

# I separately generate moments for lying, a complex behavior that is motivated by multiple treatments
import numpy as np
def calculate_lying_moments_from_decisions(voter_results, nonvoter_results, treatments):
    """Calculate 8 lying moments from individual lying decisions"""
    lying_moments = []
    short_surveys = ['0d5m', '10d5m']
    long_surveys = ['10d10m']
    
    def aggregate_lying_rate(results_dict, survey_list, lie_type):
        total_psv = 0
        total_psvl = 0
        
        for survey_type in survey_list:
            if survey_type in results_dict:
                survey_res = results_dict[survey_type]
                if 'lying_decisions' in survey_res:
                    lying_decisions = survey_res['lying_decisions'][lie_type]
                    
                    for treatment in treatments:
                        for info_type in ['NI', 'I']:
                            if (treatment in survey_res['PSV'] and 
                                info_type in survey_res['PSV'][treatment]):
                                psv = survey_res['PSV'][treatment][info_type]
                                psvl = psv * np.mean(lying_decisions)
                                total_psv += psv
                                total_psvl += psvl
        
        return total_psvl / total_psv if total_psv > 0 else 0
    
    # 8 lying moments
    lying_moments.append(aggregate_lying_rate(voter_results, short_surveys, 'none'))
    lying_moments.append(aggregate_lying_rate(voter_results, short_surveys, '5d1m'))
    lying_moments.append(aggregate_lying_rate(voter_results, long_surveys, 'none'))
    lying_moments.append(aggregate_lying_rate(voter_results, long_surveys, '8m'))
    
    lying_moments.append(aggregate_lying_rate(nonvoter_results, short_surveys, 'none'))
    lying_moments.append(aggregate_lying_rate(nonvoter_results, short_surveys, '5d1m'))
    lying_moments.append(aggregate_lying_rate(nonvoter_results, long_surveys, 'none'))
    lying_moments.append(aggregate_lying_rate(nonvoter_results, long_surveys, '8m'))
    
    return lying_moments

# Main function in this part, modeling individual decisions and generate group moments
def voteSimEndogenousVoting_vary(parameters, rand_set):
    # These are constants drawn from paper's appendix
    N = 5.4
    N_P = 10.1
    
    # Parse parameters, there are 20 in total for estimation
    params = {
        'h0_v': parameters[0], 'h0_nv': parameters[1],
        'r_v': parameters[2], 'r_nv': parameters[3], 
        'eta_v': parameters[4], 'eta_nv': parameters[5],
        'mu_s_v': parameters[6], 'mu_s_nv': parameters[7],
        'sigma_s_v': parameters[8], 'sigma_s_nv': parameters[9],
        'S_svy_v': parameters[10], 'S_svy_nv': parameters[11],
        'timeval_v': parameters[12], 'timeval_nv': parameters[13],
        'mu_sv': parameters[14], 'mu_sn': parameters[15],
        'sigma_svn': parameters[16], 'L': parameters[17],
        'mu_eps': parameters[18], 'sigma_eps': parameters[19]
    }
    
    # Generate randomness that creates variation 
    eps = params['mu_eps'] + params['sigma_eps'] * rand_set[1]
    sv = params['mu_sv'] + params['sigma_svn'] * rand_set[2]
    sn = params['mu_sn'] + params['sigma_svn'] * rand_set[3]
    
    # Voting behavior: Sv Sn stands for social value of (non) voting
    sigVal = np.maximum(sv, sn - params['L']) - np.maximum(sn, sv - params['L'])
    voted = ((sigVal * N + eps) > 0).astype(int)
    
    # Turnout rates
    turnout_control = np.mean(voted)
    
    # Survey configs: Either time incentive or momey incentive
    survey_configs = {
        '0d5m': {'base_reward': 0, 'time_adj_v': params['timeval_v']*5/60, 
                 'time_adj_nv': params['timeval_nv']*5/60},
        '10d10m': {'base_reward': 10, 'time_adj_v': 0, 'time_adj_nv': 0},
        '10d5m': {'base_reward': 10, 'time_adj_v': params['timeval_v']*5/60, 
                  'time_adj_nv': params['timeval_nv']*5/60}
    }
    
    lie_incentives = {
        'none': {'adj_v': 0, 'adj_nv': 0},
        '5d1m': {'adj_v': 5 - params['timeval_v']/60, 'adj_nv': 5 - params['timeval_nv']/60},
        '8m': {'adj_v': params['timeval_v']*8/60, 'adj_nv': params['timeval_nv']*8/60}
    }
    
    treatments = ['NF', 'F', 'FV', 'OO', 'OOV']
    
    # Here I stimulate behavior of voters and non-voters for further aggregation
    def compute_moments_for_group(voter_mask, voter_type):
        group_data = {}
        n_group = np.sum(voter_mask)
        
        if n_group == 0:
            return {key: np.zeros(len(survey_configs)) for key in treatments}
        
        h0 = params[f'h0_{voter_type}']
        r = params[f'r_{voter_type}']
        eta = params[f'eta_{voter_type}']
        s_svy = params[f'S_svy_{voter_type}']
        
        mu_s = params[f'mu_s_{voter_type}']
        sigma_s = params[f'sigma_s_{voter_type}']
        s_group = mu_s + sigma_s * rand_set[0][voter_mask]
        
        sv_group = sv[voter_mask]
        sn_group = sn[voter_mask]
        sigVal_group = sigVal[voter_mask]
        
        results = {}
        
        for survey_type, config in survey_configs.items():
            survey_results = {}
            
            time_adj = config[f'time_adj_{voter_type}']
            util_svy_only = s_group + config['base_reward'] + time_adj
            
            if voter_type == 'v':
                util_voting_q = np.maximum(sn_group - params['L'], sv_group)
            else:
                util_voting_q = np.maximum(sv_group - params['L'], sn_group)
            
            util_svy_plus_voting = util_svy_only + util_voting_q
            
            does_svy_ni = util_svy_only > -s_svy
            does_svy_i = util_svy_plus_voting > -s_svy
            
            anticip_util_ni = np.maximum(util_svy_only, -s_svy)
            anticip_util_i = np.maximum(util_svy_plus_voting, -s_svy)
            
            opts_out_oo = anticip_util_ni < 0
            opts_out_oov = anticip_util_i < 0
            
            h_star_f = np.clip(h0 + eta * anticip_util_ni, 0, 1)
            h_star_fv = np.clip(h0 + eta * anticip_util_i, 0, 1)
            
            lying_decisions = {}
            for lie_type, lie_config in lie_incentives.items():
                if voter_type == 'v':
                    would_lie = (sn_group - params['L'] + lie_config[f'adj_{voter_type}']) > sv_group
                else:
                    would_lie = (sv_group - params['L']) > (sn_group + lie_config[f'adj_{voter_type}'])
                lying_decisions[lie_type] = would_lie
            
            survey_results.update({
                'does_svy_ni': does_svy_ni, 'does_svy_i': does_svy_i,
                'opts_out_oo': opts_out_oo, 'opts_out_oov': opts_out_oov,
                'h_star_f': h_star_f, 'h_star_fv': h_star_fv,
                'lying_decisions': lying_decisions
            })
            
            results[survey_type] = survey_results
            
        return results, h0, r
    
    voter_mask = voted == 1
    nonvoter_mask = voted == 0
    
    voter_results, h0_v, r_v = compute_moments_for_group(voter_mask, 'v')
    nonvoter_results, h0_nv, r_nv = compute_moments_for_group(nonvoter_mask, 'nv')
    
    # Here I define the function that  calculate moments across voters and non-voters group
    def calculate_treatment_moments(results_dict, h0, r, group_name):
        moments = {}
        
        for survey_type in survey_configs.keys():
            res = results_dict[survey_type]
            
            ph_moments = {
                'NF': h0,
                'F': (1-r)*h0 + r*np.mean(res['h_star_f']),
                'FV': (1-r)*h0 + r*np.mean(res['h_star_fv']),
                'OO': (1-r)*h0 + r*np.mean((1-res['opts_out_oo'])*res['h_star_f']),
                'OOV': (1-r)*h0 + r*np.mean((1-res['opts_out_oov'])*res['h_star_fv'])
            }
            
            psv_nf_ni = h0 * np.mean(res['does_svy_ni'])
            psv_nf_i = h0 * np.mean(res['does_svy_i'])
            
            psv_moments = {
                'NF': {'NI': psv_nf_ni, 'I': psv_nf_i},
                'F': {
                    'NI': (1-r)*psv_nf_ni + r*np.mean(res['h_star_f']*res['does_svy_ni']),
                    'I': (1-r)*psv_nf_i + r*np.mean(res['h_star_f']*res['does_svy_i'])
                },
                'FV': {
                    'NI': (1-r)*psv_nf_ni + r*np.mean(res['h_star_fv']*res['does_svy_i']),
                    'I': (1-r)*psv_nf_i + r*np.mean(res['h_star_fv']*res['does_svy_i'])
                },
                'OO': {
                    'NI': (1-r)*psv_nf_ni + r*np.mean((1-res['opts_out_oo'])*res['h_star_f']*res['does_svy_ni']),
                    'I': (1-r)*psv_nf_i + r*np.mean((1-res['opts_out_oo'])*res['h_star_f']*res['does_svy_i'])
                },
                'OOV': {
                    'NI': (1-r)*psv_nf_ni + r*np.mean((1-res['opts_out_oov'])*res['h_star_fv']*res['does_svy_i']),
                    'I': (1-r)*psv_nf_i + r*np.mean((1-res['opts_out_oov'])*res['h_star_fv']*res['does_svy_i'])
                }
            }
            
            poo_moments = {
                'OO': h0 * r * np.mean(res['opts_out_oo']),
                'OOV': h0 * r * np.mean(res['opts_out_oov'])
            }
            
            moments[survey_type] = {
                'PH': ph_moments,
                'PSV': psv_moments, 
                'POO': poo_moments
            }
            
        return moments
    
    voter_moments = calculate_treatment_moments(voter_results, h0_v, r_v, 'v')
    nonvoter_moments = calculate_treatment_moments(nonvoter_results, h0_nv, r_nv, 'nv')
    
    # FIXED: Properly track indices
    sm = np.zeros(101)
    idx = 0
    
    # PH moments (0-29): 2 groups × 5 treatments × 3 surveys = 30
    for group_moments in [voter_moments, nonvoter_moments]:
        for treatment in treatments:
            for survey_type in survey_configs.keys():
                sm[idx] = group_moments[survey_type]['PH'][treatment]
                idx += 1
    
   
    
    # PSV moments (30-59): average across NI/I
    for group_moments in [voter_moments, nonvoter_moments]:
        for treatment in treatments:
            for survey_type in survey_configs.keys():
                psv_ni = group_moments[survey_type]['PSV'][treatment].get('NI', 0)
                psv_i = group_moments[survey_type]['PSV'][treatment].get('I', 0)
                sm[idx] = np.mean([psv_ni, psv_i])
                idx += 1
    
   
    
    # POO moments (60-71): 2 groups × 2 treatments (OO, OOV) × 3 surveys = 12
    for group_moments in [voter_moments, nonvoter_moments]:
        for treatment in ['OO', 'OOV']:
            for survey_type in survey_configs.keys():
                sm[idx] = group_moments[survey_type]['POO'][treatment]
                idx += 1
    
    
    
    # PSV by info type (72-91): 2 groups × 5 treatments × 2 info types = 20
    for group_moments in [voter_moments, nonvoter_moments]:
        for treatment in treatments:
            for info_type in ['NI', 'I']:
                values = []
                for survey_type in survey_configs.keys():
                    if info_type in group_moments[survey_type]['PSV'][treatment]:
                        values.append(group_moments[survey_type]['PSV'][treatment][info_type])
                sm[idx] = np.mean(values) if values else 0
                idx += 1
    

    # Lying moments (92-99): 8 moments
    lying_moments = calculate_lying_moments_from_decisions(
        voter_moments, nonvoter_moments, treatments
    )
    for moment in lying_moments:
        sm[idx] = moment
        idx += 1
    
    # Turnout (100): 1 moment
    sm[idx] = turnout_control
    idx += 1
    return sm

In [58]:
# SMM estimation
class SMMEstimator:
    
    def __init__(self, emp_moments, weighting_matrix, n_individuals=300000, seed=42, autosave_path=None):
        self.emp_moments = np.array(emp_moments)
        self.n_moments = len(emp_moments)
        self.n_individuals = n_individuals
        self.W = weighting_matrix
        self.autosave_path = autosave_path
        self.seed = seed 
        
        np.random.seed(seed)
        self.rand_sets = self._generate_random_sets()
        
        np.random.seed(None)
        
        self.param_config = self._setup_parameter_config()
        self.estimation_history = []
        
        
       
    #Generate consistent random draws for simulation
    def _generate_random_sets(self):
        return [
            np.random.normal(0, 1, self.n_individuals),
            np.random.normal(0, 1, self.n_individuals),
            np.random.normal(0, 1, self.n_individuals),
            np.random.normal(0, 1, self.n_individuals)
        ]
    
    # The range of each parameter can be found in paper's appendix
    def _setup_parameter_config(self):
        return {
            'names': ['h0_v', 'h0_nv', 'r_v', 'r_nv', 'eta_v', 'eta_nv', 
                     'mu_s_v', 'mu_s_nv', 'sigma_s_v', 'sigma_s_nv',
                     'S_svy_v', 'S_svy_nv', 'timeval_v', 'timeval_nv',
                     'mu_sv', 'mu_sn', 'sigma_svn', 'L', 'mu_eps', 'sigma_eps'],
            'bounds': [
                (0.20, 0.40), (0.20, 0.40),
                (0.20, 0.40), (0.20, 0.40),
                (0.0, 0.5), (0.0, 0.5),
                (-50, 0.0), (-50, 0.0),
                (0.0, 50), (0.0, 50),
                (0.0, 10), (0.0, 10),
                (0.0, 100), (0.0, 100),
                (-20, 20), (-30, 10),
                (0.0, 30), (0.0, 20),
                (-30, 100), (50, 200)
            ]
        }
    
    
    # The authors only include parameters that imply a trunout rate within (0.4,0.8)
    def generate_initial_params(self, target_turnout_range=(0.40, 0.80), max_attempts=100):
        # Generate random initial parameters bouunded by turnout rate
        for attempt in range(max_attempts):
            params = []
            for (low, high) in self.param_config['bounds']:
                params.append(np.random.uniform(low, high))
            params = np.array(params)
            
            try:
                sim_moments = voteSimEndogenousVoting_vary(params, self.rand_sets)
                implied_turnout = sim_moments[100]
                
                if target_turnout_range[0] <= implied_turnout <= target_turnout_range[1]:
                    return params
            except:
                continue
        
        print(f"Warning: Could not find parameters with turnout in {target_turnout_range} after {max_attempts} attempts")
        print("Returning random parameters without turnout constraint")
        params = []
        for (low, high) in self.param_config['bounds']:
            params.append(np.random.uniform(low, high))
        return np.array(params)
    
    
    # Define the objective function, optimal matrix as the inverse of var-cov matrix
    def criterion_function(self, parameters):
        try:
            bounds = self.param_config['bounds']
            for i, (low, high) in enumerate(bounds):
                if parameters[i] < low or parameters[i] > high:
                    return 1e10
            
            sim_moments = voteSimEndogenousVoting_vary(parameters, self.rand_sets)
            
            if np.any(~np.isfinite(sim_moments)):
                return 1e10
            
            moment_diff = self.emp_moments - sim_moments
            objective = moment_diff.T @ self.W @ moment_diff
            
            return float(objective)
            
        except Exception as e:
            print(f"Error in simulation: {e}")
            return 1e10
        
        
    # I define single starting point optimization using L-BFGS-B and Nelder-Mead methods
    def _optimize_single(self, initial_params, max_iter, verbose):
        try:
            result = optimize.minimize(
                self.criterion_function,
                initial_params,
                method='L-BFGS-B',
                bounds=self.param_config['bounds'],
                options={'disp': verbose, 'maxiter': max_iter, 'ftol': 1e-8, 'gtol': 1e-8}
            )
            
            if result.success or result.fun < 1e8:
                return result
        except Exception as e:
            if verbose:
                print(f"L-BFGS-B failed: {e}")
        
        try:
            result = optimize.minimize(
                self.criterion_function,
                initial_params,
                method='Nelder-Mead',
                options={'disp': verbose, 'maxiter': max_iter, 'xatol': 1e-8, 'fatol': 1e-8, 'adaptive': True}
            )
            return result
        except Exception as e:
            print(f"Optimization failed: {e}")
            class FailedResult:
                def __init__(self, initial_params):
                    self.fun = 1e10
                    self.x = initial_params
                    self.success = False
                    self.nfev = 0
            return FailedResult(initial_params)
        
        
    # The iteration takes huge amount of time, so below is where I can store the results and check it anytime
    def save_checkpoint(self, results_list, current_start, method, n_starts):
        #Save current progress to checkpoint file
        if self.autosave_path:
            checkpoint = {
                'results_list': results_list,
                'completed_starts': current_start,
                'total_starts': n_starts,
                'method': method,
                'best_so_far': min(results_list, key=lambda x: x.fun) if results_list else None,
                'timestamp': time.time()
            }
            try:
                with open(self.autosave_path, 'wb') as f:
                    pickle.dump(checkpoint, f)
            except Exception as e:
                print(f"Warning: Could not save checkpoint: {e}")
    
    def load_checkpoint(self):
        if self.autosave_path and os.path.exists(self.autosave_path):
            try:
                with open(self.autosave_path, 'rb') as f:
                    checkpoint = pickle.load(f)
                print(f"Loaded checkpoint: {checkpoint['completed_starts']}/{checkpoint['total_starts']} starts completed")
                return checkpoint
            except Exception as e:
                print(f"Warning: Could not load checkpoint: {e}")
        return None
    
    
# This is the main chunk of estimation
# I use everything defined above to generate multi-start optimization
    def estimate(self, method='multi_start', n_starts=100, max_iter=100000, verbose=True):
        start_time = time.time()
        
        if verbose:
            print("="*60)
            print("Starting SMM Parameter Estimation")
            print("="*60)
            print(f"Method: {method}")
            print(f"Number of moments: {self.n_moments}")
            print(f"Number of individuals: {self.n_individuals}")
            print(f"Maximum iterations: {max_iter}")
            if self.autosave_path:
                print(f"Autosave: {self.autosave_path}")
        
        results_list = []
        
        if method == 'single_start':
            initial_params = self.generate_initial_params()
            if verbose:
                print("Starting single optimization...")
            result = self._optimize_single(initial_params, max_iter, verbose)
            results_list.append(result)
            
        elif method == 'multi_start':
            if verbose:
                print(f"Running multi-start optimization with {n_starts} starts...")
            
            for i in range(n_starts):
                if verbose:
                    print(f"Start {i+1}/{n_starts}")
                
                initial_params = self.generate_initial_params()
                result = self._optimize_single(initial_params, max_iter, verbose=False)
                results_list.append(result)
                
                if verbose:
                    print(f"  Objective: {result.fun:.6f}")
                
                # Save checkpoint after each start
                self.save_checkpoint(results_list, i+1, method, n_starts)
        # I also include a staged approach, where each starting point is based on previous ones
        elif method == 'staged':
            if verbose:
                print("Running staged optimization...")
            
            coarse_results = []
            for i in range(n_starts):
                initial_params = self.generate_initial_params()
                
                if verbose:
                    print(f"Coarse optimization {i+1}/{n_starts}")
                result = self._optimize_single(initial_params, max_iter//4, verbose=False)
                coarse_results.append(result)
                
                self.save_checkpoint(coarse_results, i+1, method, n_starts)
            
            best_coarse = min(coarse_results, key=lambda x: x.fun)
            if verbose:
                print(f"Fine optimization from best (obj: {best_coarse.fun:.4f})")
            final_result = self._optimize_single(best_coarse.x, max_iter, verbose)
            results_list.append(final_result)
        else:
            raise ValueError(f"Unknown method: {method}")
        
        best_result = min(results_list, key=lambda x: x.fun)
        
        estimation_time = time.time() - start_time
        self.estimation_history.append({
            'method': method,
            'n_starts': n_starts,
            'best_objective': best_result.fun,
            'best_params': best_result.x.copy(),
            'all_results': [r.fun for r in results_list],
            'time_elapsed': estimation_time,
            'success': best_result.success
        })
        
        if verbose:
            print("="*60)
            print("Estimation Complete!")
            print("="*60)
            print(f"Best objective: {best_result.fun:.6f}")
            print(f"Time: {estimation_time:.2f} seconds ({estimation_time/60:.1f} minutes)")
            print(f"Success: {best_result.success}")
            print(f"Function evaluations: {best_result.nfev}")
            
            if len(results_list) > 1:
                obj_values = [r.fun for r in results_list]
                print(f"Objective range: {min(obj_values):.6f} - {max(obj_values):.6f}")
        
        return best_result
    
    
    # Calculate SSE
    def evaluate_fit(self, parameters=None):
        if parameters is None:
            if not self.estimation_history:
                raise ValueError("No estimation results available")
            parameters = self.estimation_history[-1]['best_params']
        
        sim_moments = voteSimEndogenousVoting_vary(parameters, self.rand_sets)
        moment_diff = self.emp_moments - sim_moments
        
        diagnostics = {
            'objective_value': float(moment_diff.T @ self.W @ moment_diff),
            'moment_differences': moment_diff,
            'max_abs_diff': float(np.max(np.abs(moment_diff))),
            'mean_abs_diff': float(np.mean(np.abs(moment_diff))),
            'rmse': float(np.sqrt(np.mean(moment_diff**2))),
            'max_rel_diff': float(np.max(np.abs(moment_diff / (self.emp_moments + 1e-10)))),
            'parameters': parameters.copy()
        }
        
        return sim_moments, diagnostics
    
    
    #Display estimation results
    def display_results(self, parameters=None):
        if parameters is None:
            if not self.estimation_history:
                print("No estimation results available")
                return None, None
            parameters = self.estimation_history[-1]['best_params']
        
        param_df = pd.DataFrame({
            'Parameter': self.param_config['names'],
            'Estimate': parameters,
            'Lower_Bound': [b[0] for b in self.param_config['bounds']],
            'Upper_Bound': [b[1] for b in self.param_config['bounds']]
        })
        
        print("\nEstimated Parameters:")
        print("="*60)
        print(param_df.to_string(index=False, float_format='%.4f'))
        
        sim_moments, diagnostics = self.evaluate_fit(parameters)
        
        print(f"\nModel Fit Diagnostics:")
        print("="*60)
        print(f"Objective: {diagnostics['objective_value']:.6f}")
        print(f"RMSE: {diagnostics['rmse']:.6f}")
        print(f"Mean Absolute Diff: {diagnostics['mean_abs_diff']:.6f}")
        print(f"Max Absolute Diff: {diagnostics['max_abs_diff']:.6f}")
        print(f"Max Relative Diff: {diagnostics['max_rel_diff']:.4f}")
        
        return param_df, diagnostics

In [81]:
#  In the paper, the authors use n_individuals=750000 and n_starts=720, which takes more than a week to finish
# Here I just use 300000 individuals and 100 starting points to illustrate the results of each run

estimator = SMMEstimator(
    emp_moments,
    weighting_matrix=W,
    n_individuals=300000,
    autosave_path='smm_checkpoint.pkl' 
)
result = estimator.estimate(method='multi_start', n_starts=100)
param_table, diagnostics = estimator.display_results()


Starting SMM Parameter Estimation
Method: multi_start
Number of moments: 101
Number of individuals: 300000
Maximum iterations: 100000
Autosave: smm_checkpoint.pkl
Running multi-start optimization with 100 starts...
Start 1/100


KeyboardInterrupt: 

In [72]:
# Here I show my results of optimal estimation
import pickle
import pandas as pd
with open('smm_checkpoint.pkl', 'rb') as f:
    checkpoint = pickle.load(f)
print(f"Process: {checkpoint['completed_starts']}/{checkpoint['total_starts']}")
print(f"Optimal objective: {checkpoint['best_so_far'].fun:.6f}\n")

param_names = ['h0_v', 'h0_nv', 'r_v', 'r_nv', 'eta_v', 'eta_nv', 
               'mu_s_v', 'mu_s_nv', 'sigma_s_v', 'sigma_s_nv',
               'S_svy_v', 'S_svy_nv', 'timeval_v', 'timeval_nv',
               'mu_sv', 'mu_sn', 'sigma_svn', 'L', 'mu_eps', 'sigma_eps']
df = pd.DataFrame({
    'Parameter': param_names,
    'Value': checkpoint['best_so_far'].x
})
print(df.to_string(index=False))

Process: 100/100
Optimal objective: 785.368755

 Parameter      Value
      h0_v   0.372406
     h0_nv   0.361436
       r_v   0.376651
      r_nv   0.292714
     eta_v   0.145226
    eta_nv   0.039576
    mu_s_v -22.282835
   mu_s_nv -38.815866
 sigma_s_v  24.442116
sigma_s_nv  37.620765
   S_svy_v   1.407816
  S_svy_nv   5.086271
 timeval_v  91.398778
timeval_nv  81.169499
     mu_sv  -6.172388
     mu_sn -23.334700
 sigma_svn  10.758911
         L  14.798783
    mu_eps -23.684616
 sigma_eps 145.616936


In [76]:
import pickle
import pandas as pd

# Load the results
with open('smm_checkpoint.pkl', 'rb') as f:
    checkpoint = pickle.load(f)

your_result = checkpoint['best_so_far']
your_params = your_result.x
your_objective = your_result.fun
print(f"My Optimal objective: {your_objective:.2f}")

# Comparison
# Load the parameters based on authors estimation
author_params = np.array([0.38,0.36,0.38,0.30,0.14,0.16,-22.6,-27.7,26.9,24.7,1.6,1.2,42.7,23.9,-3.9,-11.3,9.5,7.6,64.1,318.7]) 

param_names = ['h0_v', 'h0_nv', 'r_v', 'r_nv', 'eta_v', 'eta_nv', 
               'mu_s_v', 'mu_s_nv', 'sigma_s_v', 'sigma_s_nv',
               'S_svy_v', 'S_svy_nv', 'timeval_v', 'timeval_nv',
               'mu_sv', 'mu_sn', 'sigma_svn', 'L', 'mu_eps', 'sigma_eps']

comparison_df = pd.DataFrame({
    'Parameter': param_names,
    'Your_Estimate': your_params,
    'Author_Estimate': author_params,
    'Difference': your_params - author_params,
    'Pct_Diff': 100 * (your_params - author_params) / np.abs(author_params)
})

print("\nParameter Comparison:")
print(comparison_df.to_string(index=False, float_format='%.4f'))


My Optimal objective: 785.37

Parameter Comparison:
 Parameter  Your_Estimate  Author_Estimate  Difference  Pct_Diff
      h0_v         0.3724           0.3800     -0.0076   -1.9985
     h0_nv         0.3614           0.3600      0.0014    0.3988
       r_v         0.3767           0.3800     -0.0033   -0.8812
      r_nv         0.2927           0.3000     -0.0073   -2.4288
     eta_v         0.1452           0.1400      0.0052    3.7332
    eta_nv         0.0396           0.1600     -0.1204  -75.2651
    mu_s_v       -22.2828         -22.6000      0.3172    1.4034
   mu_s_nv       -38.8159         -27.7000    -11.1159  -40.1295
 sigma_s_v        24.4421          26.9000     -2.4579   -9.1371
sigma_s_nv        37.6208          24.7000     12.9208   52.3108
   S_svy_v         1.4078           1.6000     -0.1922  -12.0115
  S_svy_nv         5.0863           1.2000      3.8863  323.8559
 timeval_v        91.3988          42.7000     48.6988  114.0487
timeval_nv        81.1695          23.

In [89]:
# Here I try to use different n_individuals and n_starting points, and compare its results with my set up, to see if the estimation is sensitive on the choice of n_individuals and n_starting points 
import pickle
import pandas as pd

try:
    with open('checkpoint_full_720.pkl', 'rb') as f:
        checkpoint = pickle.load(f)
    
    print(f"Process: {checkpoint['completed_starts']}/{checkpoint['total_starts']}")
    print(f"Optimal SSE: {checkpoint['best_so_far'].fun:.2f}")
    print(f"Authors' SSE:160.3")
    
    # Display optimal parameters
    best_params = checkpoint['best_so_far'].x
    param_names = ['h0_v', 'h0_nv', 'r_v', 'r_nv', 'eta_v', 'eta_nv', 
                   'mu_s_v', 'mu_s_nv', 'sigma_s_v', 'sigma_s_nv',
                   'S_svy_v', 'S_svy_nv', 'timeval_v', 'timeval_nv',
                   'mu_sv', 'mu_sn', 'sigma_svn', 'L', 'mu_eps', 'sigma_eps']
    
    param_df = pd.DataFrame({
        'Parameter': param_names,
        'Value': best_params
    })
    
    print("\nOptimal Parameters:")
    print(param_df.to_string(index=False, float_format='%.4f'))
    
    # Display the best 5 SSE
    all_obj = [r.fun for r in checkpoint['results_list']]
    print(f"\nFirst 5 best SSE: {sorted(all_obj)[:5]}")
    
except FileNotFoundError:
    print("File not found")
except Exception as e:
    print(f"Error: {e}")

Process: 128/720
Optimal SSE: 789.59
Authors' SSE:160.3

Optimal Parameters:
 Parameter    Value
      h0_v   0.3723
     h0_nv   0.3580
       r_v   0.3690
      r_nv   0.3005
     eta_v   0.0471
    eta_nv   0.0977
    mu_s_v -31.6940
   mu_s_nv -39.7325
 sigma_s_v  47.9692
sigma_s_nv  38.8627
   S_svy_v   4.0562
  S_svy_nv   1.9352
 timeval_v  63.8924
timeval_nv  68.6511
     mu_sv  -6.0485
     mu_sn -11.8061
 sigma_svn   2.2204
         L  19.4944
    mu_eps  -6.6863
 sigma_eps 158.8788

First 5 best SSE: [789.58978505024, 792.4326380990643, 801.0998617341428, 806.1569423851921, 808.1385220021207]


# Comments:
(i) The estimation result is relatively stable when I change: n_individuals, n_starting points, as well as when I try different optimization
methods.

(ii) As I increase the size of n_individuals and n_starting points, the results of my estimation becomes closer to authors'results.
